**Load the HDF5 File**

In [6]:
import os
import h5py
import numpy as np
import torch
from torch.utils.data import Dataset, random_split
from torch_geometric.data import Data, InMemoryDataset
from torch_geometric.loader import DataLoader

In [32]:
class H5PointCloudDataset(Dataset):
    def __init__(self, file_path, transform=None):
        # Open the hdf5 file
        with h5py.File(file_path, 'r') as f:
            # Load the entire dataset into memory
            self.data = np.array(f['data']).astype(np.float32)    # Shape: (num_blocks, 4096, 9)
            self.labels = np.array(f['label'])   # Shape: (num_blocks, 4096)
        self.transform = transform

        # Precompute the number of unique labels (num_classes)
        self._num_classes = len(np.unique(self.labels))

    @property
    def num_classes(self):
        """Return the number of unique labels in the dataset."""
        return self._num_classes   

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self,idx):
        # Convert the 4096x9 block into a torch tensor
        block = torch.from_numpy(self.data[idx])  # Shape: (4096, 9)
        label = torch.from_numpy(self.labels[idx]).to(torch.long)  # Shape: (4096)
        
        # Slice the tensor so that:
        #   - pos gets the first 3 columns (x, y, z)
        #   - x gets the remaining 6 columns (features)
        pos = block[:, :3]
        features = block[:, 3:]
    
        # Replace the first 3 feature values of each point with zeros.
        features[:, :3] = 0

        data = Data(pos=pos, x=features, y=label)
        
        if self.transform is not None:
            data = self.transform(data)
        
        return data

In [33]:
# File path to the hdf5 dataset
script_dir = os.path.dirname(os.getcwd())
hdf5_file_path= os.path.join(script_dir, '.', 'docs', 'sim_pc_dataset.h5')

# Create the dataset
full_dataset = H5PointCloudDataset(hdf5_file_path)

# Define split sizes (e.g., 80% training and 20% validation)
total_size = len(full_dataset)
train_size = int(0.8 * total_size)
test_size = total_size - train_size

# Randomly split the dataset
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

In [9]:
print(train_dataset[0])
print(train_dataset.dataset.num_classes)

Data(x=[4096, 6], y=[4096], pos=[4096, 3])
13


In [34]:
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True,
                          num_workers=8)
test_loader = DataLoader(test_dataset, batch_size=12, shuffle=False,
                         num_workers=8)

In [68]:

# Assuming 'dataset' is your PyTorch Geometric dataset
index = 4  # Index of the data instance you want to extract

# Extract the data instance from the dataset
data_instance = train_dataset[index]

# Access the features, labels, and positions of this instance
x = data_instance.x
y = data_instance.y
pos = data_instance.pos

# Print these values
print("Features (x):", x)
print("Labels (y):", y)
print("Number of Unique Labels:", torch.unique(y))
print("Positions (pos):", pos)
print("Center Point:", torch.mean(pos, dim=0))

print(len(x))


Features (x): tensor([[0.0000, 1.0000, 0.0000, 0.1678, 0.0891, 0.0910],
        [0.0000, 1.0000, 0.0000, 0.1423, 0.0301, 0.0526],
        [0.0000, 0.0000, 1.0000, 0.1228, 0.0266, 0.2259],
        ...,
        [0.0000, 1.0000, 0.0000, 0.1464, 0.0814, 0.0910],
        [0.0000, 0.0000, 1.0000, 0.1374, 0.0087, 0.1010],
        [0.0000, 0.0000, 1.0000, 0.1115, 0.0087, 0.2324]])
Labels (y): tensor([1, 1, 2,  ..., 1, 2, 2])
Number of Unique Labels: tensor([ 1,  2, 12])
Positions (pos): tensor([[-3.8881, -5.7588,  0.0000],
        [-4.1819, -6.4146, -0.1500],
        [-4.4066, -6.4528,  0.5270],
        ...,
        [-4.1346, -5.8437,  0.0000],
        [-4.2386, -6.6528,  0.0390],
        [-4.5368, -6.6528,  0.5526]])
Center Point: tensor([-4.3428, -6.3116,  0.1144])
4096


In [7]:
all_labels = []
for data in train_dataset:
    # If data.y is a tensor, convert it to NumPy
    # If data.y is a scalar, you might need to convert it as well.
    # Here we assume data.y is a 1D tensor.
    all_labels.append(data.y.numpy())

# Concatenate all label arrays into one array
all_labels = np.concatenate(all_labels, axis=0)
unique_y = np.unique(all_labels)
print("Unique y values:", unique_y)

Unique y values: [ 0  1  2  4  5  6  7  8  9 10 11 12]


**Training with Simulated Point Cloud**

In [12]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import MLP, PointNetConv, fps, global_max_pool, radius
import torch_geometric.transforms as T
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MLP, knn_interpolate
from torch_geometric.typing import WITH_TORCH_CLUSTER
from torch_geometric.data import Data

from sklearn.linear_model import RANSACRegressor
from scipy.spatial import ConvexHull

if not WITH_TORCH_CLUSTER:
    quit("This example requires 'torch-cluster'")

In [13]:
class SAModule(torch.nn.Module):
    def __init__(self, ratio, r, nn):
        super().__init__()
        self.ratio = ratio
        self.r = r
        self.conv = PointNetConv(nn, add_self_loops=False)

    def forward(self, x, pos, batch):
        idx = fps(pos, batch, ratio=self.ratio)
        row, col = radius(pos, pos[idx], self.r, batch, batch[idx],
                          max_num_neighbors=64)
        edge_index = torch.stack([col, row], dim=0)
        x_dst = None if x is None else x[idx]
        x = self.conv((x, x_dst), (pos, pos[idx]), edge_index)
        pos, batch = pos[idx], batch[idx]
        return x, pos, batch


class GlobalSAModule(torch.nn.Module):
    def __init__(self, nn):
        super().__init__()
        self.nn = nn

    def forward(self, x, pos, batch):
        x = self.nn(torch.cat([x, pos], dim=1))
        x = global_max_pool(x, batch)
        pos = pos.new_zeros((x.size(0), 3))
        batch = torch.arange(x.size(0), device=batch.device)
        return x, pos, batch

In [14]:
class FPModule(torch.nn.Module):
    def __init__(self, k, nn):
        super().__init__()
        self.k = k
        self.nn = nn

    def forward(self, x, pos, batch, x_skip, pos_skip, batch_skip):
        x = knn_interpolate(x, pos, pos_skip, batch, batch_skip, k=self.k)
        if x_skip is not None:
            x = torch.cat([x, x_skip], dim=1)
        x = self.nn(x)
        return x, pos_skip, batch_skip

In [15]:
class Net(torch.nn.Module):
    def __init__(self, num_classes):
        super().__init__()

        # Input channels account for both `pos` and node features.
        self.sa1_module = SAModule(0.2, 0.2, MLP([3 + 6, 64, 64, 128])) # 3 (pos) + 6 (x)
        self.sa2_module = SAModule(0.25, 0.4, MLP([128 + 3, 128, 128, 256]))
        self.sa3_module = GlobalSAModule(MLP([256 + 3, 256, 512, 1024]))

        self.fp3_module = FPModule(1, MLP([1024 + 256, 256, 256]))
        self.fp2_module = FPModule(3, MLP([256 + 128, 256, 128]))
        self.fp1_module = FPModule(3, MLP([128 + 6, 128, 128, 128]))

        self.mlp = MLP([128, 128, 128, num_classes], dropout=0.5, norm=None)

        self.lin1 = torch.nn.Linear(128, 128)
        self.lin2 = torch.nn.Linear(128, 128)
        self.lin3 = torch.nn.Linear(128, num_classes)

    def forward(self, data):
        sa0_out = (data.x, data.pos, data.batch)
        sa1_out = self.sa1_module(*sa0_out)
        sa2_out = self.sa2_module(*sa1_out)
        sa3_out = self.sa3_module(*sa2_out)

        fp3_out = self.fp3_module(*sa3_out, *sa2_out)
        fp2_out = self.fp2_module(*fp3_out, *sa1_out)
        x, _, _ = self.fp1_module(*fp2_out, *sa0_out)

        return self.mlp(x).log_softmax(dim=-1)

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(num_classes=13).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

def train():
    model.train()

    total_loss = correct_nodes = total_nodes = 0
    for i, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct_nodes += out.argmax(dim=1).eq(data.y).sum().item()
        total_nodes += data.num_nodes

        if (i + 1) % 10 == 0:
            print(f'[{i+1}/{len(train_loader)}] Loss: {total_loss / 10:.4f} '
                  f'Train Acc: {correct_nodes / total_nodes:.4f}')
            total_loss = correct_nodes = total_nodes = 0

In [11]:
from torchmetrics.functional import jaccard_index
@torch.no_grad()
def test(loader):
    model.eval()
    ious = []
    
    for data in loader:
        data = data.to(device)
        outs = model(data)

        sizes = (data.ptr[1:] - data.ptr[:-1]).tolist()
        for out, y in zip(outs.split(sizes), data.y.split(sizes)):            
            iou = jaccard_index(out.argmax(dim=-1), y, 
                                num_classes=loader.dataset.num_classes, 
                                task="multiclass")
            ious.append(iou)        

    iou = torch.tensor(ious, device=device)

    mean_iou = iou.mean()
    return float(mean_iou)


In [47]:
for epoch in range(1, 21):
    train()
    iou = test(test_loader)
    print(f'Epoch: {epoch:02d}, Test IoU: {iou:.4f}')

In [ ]:
sim_checkpoint_path= os.path.join(script_dir, '.', 'docs', 'pointnet2_smartlab_sim_seg_x6_21_checkpoint.pth')

# Save model, optimizer state, and any other info needed
torch.save({
    'epoch': 21,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    #'loss': loss,
    #'test_accuracy': test_acc
}, sim_checkpoint_path)

print("Checkpoint saved successfully!")

**LoRa Fine-tuning with Pre-trained Model**

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(num_classes=13).to(device)

In [18]:
# Load the checkpoint dictionary
checkpoint = torch.load(r"C:\Users\yanpe\OneDrive - Metropolia Ammattikorkeakoulu Oy\Research\AI\scripts\checkpoints\pointnet2_s3dis_transform_seg_x6_27_checkpoint.pth", map_location=device)
# Extract the model state dictionary
model_state_dict = checkpoint['model_state_dict']

# print(model_state_dict)

In [19]:
model.load_state_dict(model_state_dict)
model.eval()

Net(
  (sa1_module): SAModule(
    (conv): PointNetConv(local_nn=MLP(9, 64, 64, 128), global_nn=None)
  )
  (sa2_module): SAModule(
    (conv): PointNetConv(local_nn=MLP(131, 128, 128, 256), global_nn=None)
  )
  (sa3_module): GlobalSAModule(
    (nn): MLP(259, 256, 512, 1024)
  )
  (fp3_module): FPModule(
    (nn): MLP(1280, 256, 256)
  )
  (fp2_module): FPModule(
    (nn): MLP(384, 256, 128)
  )
  (fp1_module): FPModule(
    (nn): MLP(134, 128, 128, 128)
  )
  (mlp): MLP(128, 128, 128, 13)
  (lin1): Linear(in_features=128, out_features=128, bias=True)
  (lin2): Linear(in_features=128, out_features=128, bias=True)
  (lin3): Linear(in_features=128, out_features=13, bias=True)
)

In [25]:
model.lin3

Linear(in_features=128, out_features=13, bias=True)

In [26]:
model.parameters()

<generator object Module.parameters at 0x0000025640FAEC00>

In [27]:
import torch.nn as nn
import loralib as lora

# Load the original model
#model = torch.load("original_model.pth")

# Freeze the entire base model
for param in model.parameters():
    param.requires_grad = False

# Apply LoRA to segmentation head
# Replace specific layers with LoRA-enabled layers
# Example: Replace a linear layer in the segmentation head
model.lin3 = lora.Linear(
    in_features=128,        # Original input dimension
    out_features=13,       # Original output dimension
    r=8,                   # Rank of the low-rank matrices (hyperparameter)
    lora_alpha=16,         # Scaling factor for LoRA weights
    merge_weights=False     # Keep LoRA separate during training
)

In [29]:
def train():
    model.train()

    total_loss = correct_nodes = total_nodes = 0
    for i, data in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        correct_nodes += out.argmax(dim=1).eq(data.y).sum().item()
        total_nodes += data.num_nodes

        if (i + 1) % 10 == 0:
            print(f'[{i+1}/{len(train_loader)}] Loss: {total_loss / 10:.4f} '
                  f'Train Acc: {correct_nodes / total_nodes:.4f}')
            total_loss = correct_nodes = total_nodes = 0

In [30]:
from torchmetrics.functional import jaccard_index
@torch.no_grad()
def test(loader):
    model.eval()
    ious = []
    
    for data in loader:
        data = data.to(device)
        outs = model(data)

        sizes = (data.ptr[1:] - data.ptr[:-1]).tolist()
        for out, y in zip(outs.split(sizes), data.y.split(sizes)):            
            iou = jaccard_index(out.argmax(dim=-1), y, 
                                num_classes=loader.dataset.num_classes, 
                                task="multiclass")
            ious.append(iou)        

    iou = torch.tensor(ious, device=device)

    mean_iou = iou.mean()
    return float(mean_iou)


In [35]:
# Train

for epoch in range(1, 21):
    train()
    iou = test(test_loader)
    print(f'Epoch: {epoch:02d}, Test IoU: {iou:.4f}')



In [ ]:
torch.save(lora.lora_state_dict(model), "pointcloud_lora_weights.pth")

In [ ]:
# Load the original model and LoRA weights
model = torch.load("original_model.pth")
lora_weights = torch.load("pointcloud_lora_weights.pth")

# Merge LoRA weights into the base model
model.load_state_dict(lora_weights, strict=False)
lora.mark_only_lora_as_trainable(model)  # Optional: Revert if needed later

# Now use `model` as a standard fine-tuned model

In [ ]:
import torch
from pointnet2 import PointNet2Seg
import loralib as lora

class PointNet2LoRA(PointNet2Seg):
    def __init__(self, num_classes, r=8, alpha=16):
        super().__init__(num_classes)
        
        # Freeze base model
        for param in self.parameters():
            param.requires_grad = False
        
        # Replace final MLP layers with LoRA
        self.seg_layers = nn.Sequential(
            lora.Linear(256, 128, r=r, lora_alpha=alpha),
            nn.ReLU(),
            lora.Linear(128, num_classes, r=r, lora_alpha=alpha)
        )

# Usage
model = PointNet2LoRA(num_classes=10, r=8)